In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

# tolerance setting
tol = 1e-3
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSNavg%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp)
solverS.output_file = 'solid_outputs/resFDSNavg%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_flux_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=inodes, value=0.0)
fluxS = InterfaceData(size=inodes, value=0.0)
tempF = InterfaceData(size=inodes, value=300.0)
tempS = InterfaceData(size=inodes, value=T_bot)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FDSNavg%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Neumann setting, study case: %i\n' % k)
flog.close()
flog = open('FDSNavg%i.log' % k, mode='a')

temps_prev = tempS.curr.copy()
fluxf_prev = fluxF.curr.copy()

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    temps_prev[:] = tempS.curr
    fluxf_prev[:] = fluxF.curr
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        tempF.backup()
        fluxF.backup()
        tempS.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface flux
        fluxF.curr[:] = solverF.get_heat_flux_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        if step > -1:
            fluxS.curr[:] = 0.5*(fluxF.curr[f2s]+fluxf_prev[f2s])
        else:
            fluxS.curr[:] = fluxF.curr[f2s]
        # update solid flux interface
        solverS.update_flux_interface(interface_tag, fluxS.curr)
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        tempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
        
        # mapping
        if step > -1:
            tempF.curr[:] = 0.5*(tempS.curr[s2f]+temps_prev[s2f])
        else:
            tempF.curr[:] = tempS.curr[s2f]
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            under_relax.determine_omega(tempF)
            under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=4.
step=1, pc_iterations=3.
step=2, pc_iterations=3.
step=3, pc_iterations=1.
step=4, pc_iterations=1.
step=5, pc_iterations=1.
step=6, pc_iterations=1.
step=7, pc_iterations=1.
step=8, pc_iterations=1.
step=9, pc_iterations=1.
step=10, pc_iterations=1.
step=11, pc_iterations=1.
step=12, pc_iterations=1.
step=13, pc_iterations=1.
step=14, pc_iterations=1.
step=15, pc_iterations=1.
step=16, pc_iterations=1.
step=17, pc_iterations=1.
step=18, pc_iterations=1.
step=19, pc_iterations=1.
step=20, pc_iterations=1.
step=21, pc_iterations=1.
step=22, pc_iterations=1.
step=23, pc_iterations=1.
step=24, pc_iterations=1.
step=25, pc_iterations=1.
step=26, pc_iterations=1.
step=27, pc_iterations=1.
step=28, pc_iterations=1.
step=29, pc_iterations=1.
step=30, pc_iterations=1.
step=31, pc_iterations=1.
step=32, pc_iterations=1.
step=33, pc_iterations=1.
step=34, pc_iterations=1.
step=35, pc_iterations=1.
step=36, pc_iterations=1.
step=37, pc_iterations=1.
step=38, pc_iterations

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFDSN_avg%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})

[0.88421727 0.89253021 0.89841661 0.90316078 0.9071311  0.9104409
 0.91334061 0.91589226 0.91818921 0.92031453 0.92222407 0.9239959
 0.92562253 0.92713788 0.92853754 0.92984484 0.93106037 0.93220402
 0.93326926 0.93426352 0.93519575 0.93606732 0.93688747 0.93765261
 0.93836881 0.93903816 0.93966881 0.94025713 0.94080989 0.94132631
 0.94180772 0.94225932 0.94267974 0.94307173 0.94343952 0.94377973
 0.94409641 0.94438929 0.94466264 0.94491423 0.94514662 0.94536142
 0.94555729 0.94573637 0.94589924 0.94604635 0.9461786  0.94629616
 0.94639951 0.94649016 0.94656647 0.94663044 0.94668106 0.94671916
 0.94674524 0.94675804 0.94675979 0.94674821 0.94672458 0.94668814
 0.94663887 0.94657756 0.94650194 0.94641323 0.94630907 0.94619119
 0.94605833 0.9459084  0.94574288 0.945559   0.94535619 0.94513491
 0.94489142 0.94462776 0.94433926 0.94402617 0.94368353 0.94331403
 0.94291322 0.94247657 0.94200534 0.94149086 0.94093301 0.94033048
 0.93967146 0.93895512 0.93816996 0.93731427 0.93637336 0.935332